In [1]:
%%javascript
$('#appmode-leave').hide();
$('#copy-binder-link').hide();
$('#visit-repo-link').hide();

<IPython.core.display.Javascript object>

# Working notebooks

In [2]:
import ipywidgets as ipw
import json
import random
import time
import pandas as pd
import os
import webbrowser
import math
import numpy as np
from IPython.display import display, Markdown, FileLink, clear_output

In [3]:
from IPython.display import Javascript
import glob as glob
import nbformat as nbf

label_layout = ipw.Layout(width='300px')

##########
def launchNotebook(filename):
    text = "   var name_of_the_notebook = '" + filename + "'"
    vv="""
    var url = window.location.href.split('/')
    var newurl = url[0] + '//'
    for (var i = 1; i < url.length - 1; i++) {
        console.log(url[i], newurl)
        newurl += url[i] + '/'
    }
    newurl += name_of_the_notebook
    window.open(newurl)
    """
    text = text + vv
    display(Javascript(text))

def openNewNotebook(btn):
    if os.path.exists(notebookeName.value):
        print("Filename exists - Please select a different name")
        return
    
    nb = nbf.v4.new_notebook()
    text = """# Click 'Edit App' to start coding"""

    code = """\
# python packages
import pandas as pd # Dataframes and reading CSV files
import numpy as np # Numerical libraries
import matplotlib.pyplot as plt # Plotting library
from lmfit import Model # Least squares fitting library
from scipy.optimize import curve_fit # Alternative curve fittting library"""

    nb['cells'] = [nbf.v4.new_markdown_cell(text),
                   nbf.v4.new_code_cell(code)]

    with open(notebookeName.value, 'w') as f:
        nbf.write(nb, f)

    launchNotebook(notebookeName.value)

##########
listOfFiles = []
files = glob.glob1("./","*.ipynb")
for f in files:
    if f == "virtualExperiment.ipynb" or f == "crystalViolet.ipynb":
        continue
    listOfFiles.append(f)

def dropdown_filesHandler(change):
    for i in range(0,len(listOfFiles)):
        if listOfFiles[i] == change.new:
            oldNotebookeName[0] = listOfFiles[i]

def createMenuFiles(data):
    option_list = ["Choose one"]
    option_list.extend(data)

    dropdown = ipw.Dropdown(description="", options=option_list, layout=ipw.Layout(width="300px"))
    dropdown.observe(dropdown_filesHandler, names='value')

    return dropdown

##########
oldNotebookeName = ["None"]
def openOldNotebook(btn):
    if oldNotebookeName[0] == "None":
        print("Please select a filename")
    else:
        launchNotebook(oldNotebookeName[0])

##########
actions0 = []

notebookeName = ipw.Text("Empty.ipynb")

btn_new = ipw.Button(description="Create a new notebook", layout=label_layout)
btn_new.on_click(openNewNotebook)

btn_old = ipw.Button(description="Open an old notebook", layout=label_layout)
btn_old.on_click(openOldNotebook)

actions0.append(ipw.HBox([btn_new,notebookeName]))
actions0.append(ipw.HBox([btn_old,createMenuFiles(listOfFiles)]))
ipw.VBox(actions0)


# Crystal Violet Virtual Experiment

In [4]:
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

In [5]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
   return false;
}

<IPython.core.display.Javascript object>

In [6]:
with open(".lab.json") as infile:
    jsdata = json.load(infile)

params = jsdata["cv"]

In [7]:
t = int( time.time() * 1000.0 )
random.seed( ((t & 0xff000000) >> 24) +
             ((t & 0x00ff0000) >>  8) +
             ((t & 0x0000ff00) <<  8) +
             ((t & 0x000000ff) << 24)   )

params["NaOH"] = 0.5 * random.gauss(1,params["error"])
params["CV"] = 2.5e-5 * random.gauss(1,params["error"])

In [10]:
def run_experiment():

    tt = random.gauss(float(T.value),params["et"])
    x0 = float(v0.value) + random.gauss(0,params["ev"]) # CV
    x1 = float(v1.value) + random.gauss(0,params["ev"]) # OH
    x2 = float(v2.value) + random.gauss(0,params["ev"]) # H2O

    lnk = params["A"] - params["B"]/params["R"]/(tt+273.15)
    kr = math.exp(lnk)

    vt = x0 + x1 + x2
    ccv = params["CV"] * x0 / vt
    coh = params["NaOH"] * x1 / vt
    
    kd = kr * math.pow(coh,params["beta"])

    Abs = params["A0"]*ccv
        
    res = pd.DataFrame(columns=["#Time [s]" , "Absorbance"])
    for i in range(2, params["nTime"]):
        var_list = []
        var_list.append(i)
        
        expVal = Abs * math.exp(-kd*i) + abs(random.gauss(0,params["error"]) + 0.008)
        var_list.append(expVal)
        res.loc[len(res)] = var_list
    
    res.to_csv(respath.value, index=False, mode='a')
 
    f = open(respath.value , "a")
    f.write("!CV  = %a\n" % v0.value)
    f.write("!OH  = %a\n" % v0.value)
    f.write("!H2O = %a\n" % v0.value)
    f.write("!H2O = %a\n" % T.value)

    local_file = FileLink(respath.value, result_html_prefix="Click here to download: ")
    with out_P:
        display(local_file)
        display(res.tail(params["nTime"]))


In [9]:
out_Error = ipw.Output()
out_P = ipw.Output()

# output filename
# fileName = "results.csv"
respath = ipw.Text("results.csv")

v0 = ipw.Text("10")
box00 = ipw.Box([ipw.Label("2.5x10$^{-5}$M stock solution of CV (ml)",layout=label_layout),v0])
v1 = ipw.Text("10")
box01 = ipw.Box([ipw.Label("0.5 M stock solution of NaOH (ml)",layout=label_layout),v1])
v2 = ipw.Text("10")
box02 = ipw.Box([ipw.Label("Deionised water (ml)",layout=label_layout),v2])
T = ipw.Text("25")
box03 = ipw.Box([ipw.Label("Temperature ($^\circ$C)",layout=label_layout),T])

def reset(btn):
    if os.path.exists(respath.value):
        os.remove(respath.value)
    
    with out_Error:
        out_Error.clear_output()
        
    with out_P:
        out_P.clear_output()
        
    clear_output()
    create_ipw()

def calc(btn):
    out_P.clear_output()
    run_experiment()

# interactive buttons ---
btn_calc = ipw.Button(description="Perform Experiment", layout=ipw.Layout(width="150px"))
btn_calc.on_click(calc)

btn_reset = ipw.Button(description="Reset Experiment", layout=ipw.Layout(width="150px"))
btn_reset.on_click(reset)

# -- output widgets
def create_ipw():
    rows = []

    label_layout = ipw.Layout(width='300px')

    rows.append(ipw.HBox([ipw.Label('Output filename : ',layout=label_layout),respath]))
    rows.append(ipw.HBox([box00]))
    rows.append(ipw.HBox([box01]))
    rows.append(ipw.HBox([box02]))
    rows.append(ipw.HBox([box03]))

    rows.append(ipw.HBox([btn_calc,btn_reset]))
    rows.append(ipw.HBox([out_Error]))
    rows.append(ipw.HBox([out_P]))
    display(ipw.VBox(rows))

create_ipw()
